IMPORTS & PREPRODUCTION


In [1]:
import time
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
    
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


GPU found


In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hyunseokerikjung. Use `wandb login --relogin` to force relogin


True

Data Processing

In [3]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [4]:
aug_df = pd.read_csv('augmented_emnist.csv')
aug_df.rename(columns={'25':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

train_y = np.array(aug_df.pop('Target'))
train_x = aug_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)


In [5]:

test_df = pd.read_csv('emnist-byclass-test.csv/emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

test_y = np.array(test_df.pop('Target'))
test_x = test_df

test_x = test_x / 255.0
test_x = test_x.values.reshape([-1, 28, 28, 1])
test_y = keras.utils.to_categorical(test_y)


In [6]:
hand_df = pd.read_csv('our_handmade_dataset.csv')
hand_df.rename(columns={'0':'Target'}, inplace=True)

# hand_df=pd.merge(hand_df, data, how='left', on='Target')
# hand_df=hand_df.drop(['Target','ASCII'],axis=1)
# hand_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

hand_y = np.array(hand_df.pop('Target'))
hand_x = hand_df

hand_x = hand_x / 255.0
hand_x = hand_x.values.reshape([-1, 28, 28, 1])
hand_y = keras.utils.to_categorical(hand_y)


In [7]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode="constant",
    cval=0
)


MODELS

Original LeNet_5

In [10]:
keras.backend.clear_session()

lenet_5_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])
lenet_5_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [11]:
lenet_5_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

wandb.init(project="new_data_test", entity="hyunseokerikjung")
lenet_5_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, WandbCallback()])

lenet_5_model.evaluate(test_x, test_y)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
13830/13830 [==============================] - 47s 3ms/step - loss: 0.8872 - accuracy: 0.6666 - val_loss: 0.7262 - val_accuracy: 0.7025 - _timestamp: 1653406599.0000 - _runtime: 60.0000
Epoch 2/100
13830/13830 [==============================] - 45s 3ms/step - loss: 0.6869 - accuracy: 0.7161 - val_loss: 0.6892 - val_accuracy: 0.7154 - _timestamp: 1653406644.0000 - _runtime: 105.0000
Epoch 3/100
13830/13830 [==============================] - 46s 3ms/step - loss: 0.6532 - accuracy: 0.7259 - val_loss: 0.6566 - val_accuracy: 0.7233 - _timestamp: 1653406690.0000 - _runtime: 151.0000
Epoch 4/100
13830/13830 [==============================] - 44s 3ms/step - loss: 0.6331 - accuracy: 0.7320 - val_loss: 0.6462 - val_accuracy: 0.7275 - _timestamp: 1653406735.0000 - _runtime: 196.0000
Epoch 5/100
13830/13830 [==============================] - 44s 3ms/step - loss: 0.6200 - accuracy: 0.7354 - val_loss: 0.6369 - val_accuracy: 0.7318 - _timestamp: 1653406779.0000 - _runtime: 240.0000
Epoch 

[0.713187038898468, 0.7460497617721558]

In [12]:
keras.backend.clear_session()

lenet_5_model_img_gen = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])
lenet_5_model_img_gen.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [13]:
lenet_5_model_img_gen.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

wandb.init(project="new_data_test", entity="hyunseokerikjung")
lenet_5_model_img_gen.fit(aug.flow(train_x, train_y), epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, WandbCallback()])

lenet_5_model_img_gen.evaluate(test_x, test_y)

accuracy,▁▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▅▆▆▆▇▇▆▇▇▇▇▇▇██▇▇▇▇███▇▇███▇███
val_loss,█▆▄▃▂▂▂▂▂▂▂▁▂▁▂▁▁▁▂▂▁▁▁▁▂▁▂▂▂▂▂▁▁
accuracy,0.76539
best_epoch,22
best_val_loss,0.61369
epoch,32
loss,0.53575
val_accuracy,0.74045


Epoch 1/100
13830/13830 [==============================] - 72s 5ms/step - loss: 1.0988 - accuracy: 0.6087 - val_loss: 0.7699 - val_accuracy: 0.6932 - _timestamp: 1653408069.0000 - _runtime: 83.0000
Epoch 2/100
13830/13830 [==============================] - 71s 5ms/step - loss: 0.8329 - accuracy: 0.6727 - val_loss: 0.7169 - val_accuracy: 0.7050 - _timestamp: 1653408140.0000 - _runtime: 154.0000
Epoch 3/100
13830/13830 [==============================] - 72s 5ms/step - loss: 0.7951 - accuracy: 0.6838 - val_loss: 0.6835 - val_accuracy: 0.7177 - _timestamp: 1653408212.0000 - _runtime: 226.0000
Epoch 4/100
13830/13830 [==============================] - 72s 5ms/step - loss: 0.7736 - accuracy: 0.6882 - val_loss: 0.6808 - val_accuracy: 0.7163 - _timestamp: 1653408284.0000 - _runtime: 298.0000
Epoch 5/100
13830/13830 [==============================] - 71s 5ms/step - loss: 0.7603 - accuracy: 0.6928 - val_loss: 0.6796 - val_accuracy: 0.7153 - _timestamp: 1653408355.0000 - _runtime: 369.0000
Epoch 

[0.6026073098182678, 0.7416223883628845]

In [14]:
keras.backend.clear_session()

my_lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(16, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

my_lenet_5_selu_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        25664     
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 32)        18464     
                                                                 
 batch_normalization_2 (Batc  (None, 22, 22, 32)       1

In [15]:
my_lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
  
my_lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, WandbCallback()])

my_lenet_5_selu_model.evaluate(test_x, test_y)

Epoch 1/100
13830/13830 [==============================] - 49s 4ms/step - loss: 1.1038 - accuracy: 0.6280 - val_loss: 0.9673 - val_accuracy: 0.6450 - _timestamp: 1653411219.0000 - _runtime: 3233.0000
Epoch 2/100
13830/13830 [==============================] - 49s 4ms/step - loss: 0.7763 - accuracy: 0.6993 - val_loss: 0.7697 - val_accuracy: 0.6999 - _timestamp: 1653411268.0000 - _runtime: 3282.0000
Epoch 3/100
13830/13830 [==============================] - 49s 4ms/step - loss: 0.7315 - accuracy: 0.7109 - val_loss: 0.7919 - val_accuracy: 0.6919 - _timestamp: 1653411317.0000 - _runtime: 3331.0000
Epoch 4/100
13830/13830 [==============================] - 48s 4ms/step - loss: 0.7088 - accuracy: 0.7174 - val_loss: 0.7055 - val_accuracy: 0.7173 - _timestamp: 1653411365.0000 - _runtime: 3379.0000
Epoch 5/100
13830/13830 [==============================] - 47s 3ms/step - loss: 0.6942 - accuracy: 0.7206 - val_loss: 0.7237 - val_accuracy: 0.7133 - _timestamp: 1653411412.0000 - _runtime: 3426.0000


[0.5789974331855774, 0.7883375287055969]

In [16]:
now = time.time()
now = time.localtime(now)
my_lenet_5_selu_model.save("my_lenet5_emnist_model_%2d%02d%02d%02d%02d.h5" %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min), save_format="h5")